# Evaluation

In [1]:
import numpy as np

import torch
import torch.nn as nn

import torchvision.transforms as T

from torchvision.models import ResNet
from time_shift_dataset import TimeShiftDataset

from tqdm import tqdm

In [2]:
backbone: ResNet = torch.load('./models/backbone_time_shift_PRX_60.pth')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
_ = backbone.to(device)

transform = T.Compose([
        T.Resize(128),
        T.ToTensor(),
        T.Grayscale()
    ])

train_dataset = TimeShiftDataset('../datasets/hand', transform=transform, train=True)
test_dataset = TimeShiftDataset('../datasets/hand', transform=transform, train=False)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=8,
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
)

cuda


## Evaluation: Linear Classification

In [3]:
class LinearEval(nn.Module):
    """Some Information about LinearEval"""
    def __init__(self, backbone: nn.Module):
        super(LinearEval, self).__init__()
        self.backbone = backbone
        self.linear = nn.Linear(in_features=512, out_features=10, bias=True)

    def forward(self, x):
        x = self.backbone(x)
        x = torch.flatten(x, start_dim=1)
        x = self.linear(x)

        return x

In [4]:
eval_model = LinearEval(backbone=backbone).to(device)
for param in eval_model.backbone.parameters():
    param.requires_grad = False

In [5]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(eval_model.parameters(), lr=0.06, momentum=0.8)

In [6]:
def test_model(model, testloader):
    wrongly_classified = 0
    for i, data in enumerate(testloader, 0):
        total = len(data[0])
        inputs, labels = data
        inputs,labels = inputs.to(device), labels.to(device)

        with torch.no_grad():
            preds = model(inputs).argmax(dim=1)

        wrong = total - (preds == labels).sum()
        wrongly_classified += wrong

    return wrongly_classified / len(testloader)

In [7]:
def train(model, train_loader, test_loader, device):
    # loop over the dataset multiple times
    for epoch in range(30):
        running_loss = 0.0
        for i, data in enumerate(tqdm(train_loader), 0):
            inputs, _, labels, _ = data
            labels = nn.functional.one_hot(labels).float()
            inputs, labels = inputs.to(device), labels.to(device)


            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = eval_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print('Epoch: {}, Loss: {}, Test error: {}'.format(epoch, running_loss, test_model(eval_model, test_loader)))

    print('Finished Training')

In [9]:
train(eval_model, train_dataloader, test_dataloader, device)

  0%|          | 0/39 [00:05<?, ?it/s]


KeyboardInterrupt: 